In [ ]:
#Importing libraries
import re
import tweepy 
from textblob import TextBlob
import pandas as pd
import nltk
import string
import sqlite3
from geopy.geocoders import Nominatim
import csv

In [ ]:
#Enter the following details of your twitter developer account
TWITTER_APP_KEY= 'Your App Key'
TWITTER_APP_SECRET= 'Your App Secret'
TWITTER_KEY= 'Your Token Key'
TWITTER_SECRET= 'Your Token Secret'

In [ ]:
#Authentication of account
auth = tweepy.OAuthHandler(TWITTER_APP_KEY, TWITTER_APP_SECRET)
auth.set_access_token(TWITTER_KEY, TWITTER_SECRET)
api = tweepy.API(auth)

In [ ]:
#Creating database for local storage
conn = sqlite3.connect("tweet.db")
c = conn.cursor()

In [ ]:
#Creating a table
c.execute("""CREATE TABLE tweets (
            State text,
            Positive_Count integer,
            Negative_Count integer,
            Neutral_Count integer
            )""")
conn.commit()

In [ ]:
#Initializing all states and union territories data with 0 in database
c.execute("""INSERT INTO tweets VALUES ('State_Name',0,0,0)""")
conn.commit()

In [ ]:
#For storing data in csv file from database
header = []
for column in c.execute('PRAGMA table_info("tweets")'):
    header.append(column[1])

In [ ]:
#For checking the location using latitude and longitude
geolocator = Nominatim(user_agent="geoapiExercises")
places = api.search_geo(query="India", granularity="country")
place_id = places[0].id
tweets = api.search_tweets(q="place:%s" % place_id)

In [ ]:
count_pos = 0
count_neg = 0
count_neu = 0
global ls
ls = []

In [ ]:
#All the functionalities are in on_status function
class StreamListener(tweepy.Stream):
    def on_status(self, status):
        #hasattr used so that retweeted tweets are not included
        if hasattr (status, 'retweeted_status'):
            return
        if status.place == None:
            return
        country = str(status.place.country)
        #Including tweets from India
        if country!='India':
            return
        longitude = str(status.place.bounding_box.coordinates[0][0][0])
        latitude = str(status.place.bounding_box.coordinates[0][0][1])
        location = geolocator.reverse(latitude+","+longitude)
        try:
            #Finding state in India using latitude and longitude
            state = location.raw['address']['state']
        except:
            return
        tweet = status.text
        if status.id in ls:
            #To check the same tweets do not appear
            return
        ls.append(status.id)
        #Cleaning tweets
        cleaned_tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
        #Using TextBlob for sentiment analysis
        analysis = TextBlob(cleaned_tweet)
        #Updating data in database with different polarities 
        if analysis.sentiment.polarity > 0:
            c.execute("""SELECT * FROM tweets WHERE State = ? """,(state,))
            count_pos = c.fetchall()[0][1]
            conn.commit()
            count_pos+=1
            c.execute("""UPDATE tweets SET Positive_Count = ? WHERE State = ? """,(count_pos,state))
            conn.commit()
        elif analysis.sentiment.polarity == 0:
            c.execute("""SELECT * FROM tweets WHERE State = ? """,(state,))
            count_neu = c.fetchall()[0][2]
            conn.commit()
            count_neu+=1
            c.execute("""UPDATE tweets SET Neutral_Count = ? WHERE State = ? """,(count_neu,state))
            conn.commit()
        else:            
            c.execute("""SELECT * FROM tweets WHERE State = ? """,(state,))
            count_neg = c.fetchall()[0][3]
            conn.commit()
            count_neg+=1
            c.execute("""UPDATE tweets SET Negative_Count = ? WHERE State = ? """,(count_neg,state))
            conn.commit()
        #Making a csv file from database data
        df = pd.DataFrame(columns=header)
        for raw in c.execute('SELECT * FROM tweets ORDER BY State'):
            series = pd.Series(list(raw), index=df.columns)
            df = df.append(series, ignore_index=True)
        df.to_csv('tweet.csv')
        #Now uploaded this csv to google sheets and 
        #updating the data in google sheet according to changes in local csv file using Couplor.io and 
        #finally making a choropleth map of India using datawrapper
        
    def on_error(self, status_code):
        if status_code == 420:
            return False

In [ ]:
#Finally using the following starting real time streaming using filter of geographic location
stream_listener = StreamListener(TWITTER_APP_KEY, TWITTER_APP_SECRET,TWITTER_KEY, TWITTER_SECRET)
stream_listener.filter(track=place_id)